# Netzwerke modellieren mit pandapipes

Diese Einführung stellt die von pandapipes genutzten Datenstrukturen vor und zeigt, wie mit Hilfe der pandapipes API Netzwerke erstellt werden können. Das folgende Beispiel enthält die gängigsten Komponenten, die im Rahmen von pandapipes eingesetzt werden.

<img src="pics/simple_network-1.png">

Die pandapipes-Umgebung basiert auf der Python-Bibliothek "pandas". Ein Netzwerk enthält Tabellen für jeden Komponententyp, der eingesetzt wird. Die Zeilen der Tabellen entsprechen den tatsächlich im Netzwerk vorhandenen Komponenten. Jede Zeile in einer Komponententabelle repräsentiert also eine Komponente des jeweiligen Typs. Die Spalten entsprechen den Eigenschaften des Komponententyps.

Durch das Ausführen der folgenden Code-Zellen des Notebooks werden die für das betrachtete Netz benötigten Komponententabellen erstellt. Wenn Sie weitere Informationen zu den einzelnen Parametern der Komponenten benötigen, so finden Sie diese in der pandapipes-Dokumentation unter Punkt "Datastructure and Components".


### Leeres Netzwerk

Zunächst wird pandapipes importiert und ein leeres Netzwerk erzeugt:

Für jede Berechnung ist es wichtig, ein Transportmedium auszuwählen. Obwohl auch eigene Fluide definiert werden können, greifen wir im Rahmen dieses Tutorials auf die Fluidbibliothek zurück. Ein Fluid wird über die Angabe des entsprechenden Namens aus der Bibliothek geladen. Namenskürzel sind z.B.  <span style="color:green">hgas</span>, <span style="color:green">lgas</span>,  <span style="color:green">hydrogen</span>,
<span style="color:green">water</span>, und <span style="color:green">air</span>.

Da wir ein Erdgasnetz erzeugen wollen, wählen wir <span style="color:green">lgas</span> aus der Bibliothek aus.

In [ ]:
import pandapipes as pp

In [ ]:
net = pp.create_empty_network(fluid="lgas") # create an empty network

An diesem Punkt enthält das Netz noch keinerlei Komponenten. Es dient aber als ein Container für die Komponententabellen und netzwerkspezifische Parameter.

In [ ]:
net

Die <span style="color:blue">create</span>-Funktionen werden nun eingesetzt, um neue Komponenten zu erzeugen. Diese werden automatisch den entsprechenden Tabellen hinzugefügt. 

### Junctions

<img src="pics/simple_network-junc.png">

In einem ersten Schritt werden die Knotenpunkte des Netzwerks erstellt, zwischen denen Rohrleitungen und andere Komponenten verlaufen. Diese Knotenpunkte werden "junctions" genannt. Insgesamt werden davon sechs benötigt. Für jeden Knotenpunkt wird zudem ein Anfangswert benötigt. Bei diesem handelt es sich um einen initialen Schätzwert, der im Laufe der Berechnung angepasst und am Ende durch das Berechnungsergebnis ersetzt wird. Der Anfangswert beträgt 1.0 bar.

Außerdem wird eine Temperatur für das Fluid definiert. Diese wird für die hydraulischen Berechnungen benötigt.

Das Vergeben von Namen ist optional, erhöht aber die Lesbarkeit der Ergebnisse. 

Diese Werte werden als Parameter der create_junction-Funktion übergeben.

In [ ]:
junction1 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Connection to External Grid")
junction2 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Junction 2")
junction3 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Junction 3")
junction4 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Junction 4")
junction5 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Junction 5")
junction6 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Junction 6")

Die definierten Knoten befinden sich jetzt im Netz-Container und können abgerufen werden. Einigen Parametern, die beim Funktionsaufruf nicht explizit erwähnt worden sind, wurden Standardwerte zugewiesen.

In [ ]:
net.junction # show junction table

Alle create-Funktionen geben den pandapipes-Index des Elements zurück, welches erzeugt wurde. Beispielsweise entspricht die Variable "junction1" dem Index des Knoten mit dem Namen "Connection to External Grid". Der Index ist entsprechend 0.

In [ ]:
junction1

In [ ]:
net.junction.loc[junction1]

Wir nutzen diese Variablen im Folgenden, um Komponenten, wie z.B. Rohre, mit den Knoten zu verbinden.

### External Grid

<img src="pics/simple_network-ext_grid.png">

Ein External Grid repräsentiert eine mögliche Schnittstelle zum einem Netzbereich, der vom pandapipes-Modell nicht erfasst wird. Mathematisch gesehen wird der Druck an dem Knoten, an dem das External Grid angeschlossen ist, während der Berechnung nicht mehr angepasst. Sie teilen dem Programm damit mit, dass Ihnen der Wert des Druckes an dieser Stelle bereits bekannt ist. Für eine erfolgreiche Berechnung ist die Bekanntgabe eines festen Druckwertes an wenigstens einer Position im Netz unbedingt notwendig. Das External Grid erfüllt mit diesen Eigenschaften die Funktions eines slack-Knotens.

in unserem Beispiel wird das External Grid mit der Junction-Variable junction1 verbunden. Der Druck beträgt 1.1 bar. In der späteren Auswertung wird der Druck, den Sie an junction1 abrufen, folglich ebenfalls 1.1 bar betragen.


In [ ]:
medium_pressure_grid = pp.create_ext_grid(net, junction=junction1, p_bar=1.1, t_k=293.15, name="Grid Connection")

net.ext_grid # show external grid table

### Rohre

Das Netzwerk enthält 5 Rohre, die zwischen je zwei Knoten verlaufen. Die Verbindungspunkte und die jeweiligen Rohrlängen sind im Diagramm zu sehen:

<img src="pics/simple_network-pipes.png">

Alle Rohre haben einen Durchmesser von 300 mm. 

In [ ]:
pipe1 = pp.create_pipe_from_parameters(net, from_junction=junction1, to_junction=junction2, length_km=10, diameter_m=0.3, name="Pipe 1")
pipe2 = pp.create_pipe_from_parameters(net, from_junction=junction2, to_junction=junction3, length_km=2, diameter_m=0.3, name="Pipe 2")
pipe3 = pp.create_pipe_from_parameters(net, from_junction=junction2, to_junction=junction4, length_km=2.5, diameter_m=0.3, name="Pipe 3")
pipe4 = pp.create_pipe_from_parameters(net, from_junction=junction3, to_junction=junction5, length_km=1, diameter_m=0.3, name="Pipe 4")
pipe5 = pp.create_pipe_from_parameters(net, from_junction=junction4, to_junction=junction6, length_km=1, diameter_m=0.3, name="Pipe 5")

Damit sieht die Tabelle der Rohrkomponenten wie folgt aus:

In [ ]:
net.pipe # show pipe table

Die Parameter from_junction und to_junction geben die Orientierung des Rohrs an. Wenn die Flussrichtung tatsächlich aus Richtung from_junction in Richtung to_junction verläuft, ist das Vorzeichen der berechneten Strömungsgeschwindigkeit positiv. Andernfalls nimmt die Strömungsgeschwindigkeit negative Werte an.

### Ventil

Es gibt ein Ventil zwischen Junction 5 und Junction 6. Dieses ist geöffnet, sodass das Gas das Ventil passieren kann.

<img src="pics/simple_network-valve.png">

In [ ]:
valve = pp.create_valve(net, from_junction=junction5, to_junction=junction6, diameter_m=0.3, opened=True)

Wie man sieht, wird standardmäßig kein Druckverlustkoeffizient für das Ventil angegeben. Außerdem haben Ventile keine Länge. Der Druckverlustkoeffizient kann bei Bedarf modifiziert werden.

In [ ]:
net.valve # show valve table

### Senke

<img src="pics/simple_network-sink.png">

Mit Hilfe einer Senke kann eine konstante Abnahme, wie bspw. einer KWK-Anlage, definiert werden. An dieser Stelle wird eine sink-Komponente mit einem Massestrom in Höhe von 545 g/s definiert. Senken und Quellen werden immer einem bestimmten Knoten zugeordnet. Das Vorzeichen des Massestroms ist positiv, wenn ein Massestrom aus dem System entnommen wird.


In [ ]:
sink = pp.create_sink(net, junction=junction4, mdot_kg_per_s=0.545, name="Sink 1")
net.sink

### Quelle

<img src="pics/simple_network-source.png">

Die Source-Komponente wird genutzt, um einen Wärme- oder Massestrom in das Netz einzuspeisen. Ein denkbarer Fall ist der einer Biogas- oder P2G-Anlage, welche direkt an das Gasnetz angeschlossen ist. 

In diesem Beispiel nehmen wir an, dass eine Biogasanlage direkt ins Gasnetz einspeist. Die Anlage liefert einen konstanten Strom in Höhe von 234 g/s. Aus Sicht der Source-Komponente ist das Vorzeichen positiv, wenn ins System eingespeist wird.

In [ ]:
source = pp.create_source(net, junction=junction3, mdot_kg_per_s=0.234, name="Source 1")

In [ ]:
net.source

Die Modellierung des Netzes ist damit abgeschlossen. Abschließend können alle Komponenten des Netzes angezeigt werden.

In [ ]:
net

Die eigentliche Berechnung wird mit dem `pipeflow`-Kommando gestartet:

In [ ]:
pp.pipeflow(net)

Ergebnisse werden in Tabellen mit dem Präfix `res_...` gespeichert. Auch diese Tabellen sind nach der Berechnung im  `net`-Container abgelegt. 

In [ ]:
net # result tables have been added to the net 

In [ ]:
net.res_junction # calculated pressure and temperature at junctions

In [ ]:
net.res_pipe  # velocities, mass flows through pipes and other results

### Lessons Learned

Nach dem Durcharbeiten des Tutorials sollten Sie:
    
* wissen, welche Schritte vor dem Start einer Berechnung durchzuführen sind.
- einen Überblick über die gängigsten pandapipes-Komponenten bekommen haben.
- ein Gefühl dafür bekommen haben, wie man bei der Modellierung mit dem kommandozeilenbasierten Programm vorgeht und 
  wie man Parameter von neuen Komponenten modifizieren kann.
- wissen, mit welchem Befehl eine Berechnung gestartet wird.
- wissen, inwieweit sich die Struktur des Netz-Containers nach erfolgter Berechnung verändert und wie Sie auf Berechnungsergebnisse zugreifen können.